In [17]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import metrics
from read_tfrecord import *
from mtcnn_model import Rnet,cls_ohem,cal_accuracy,bbox_ohem, landmark_ohem
from tqdm import tqdm

In [18]:
data_path = "data/24/train_RNet_landmark.tfrecord_shuffle"

In [19]:
def load_pokemon(mode='train'):
    """ 加载pokemon数据集的工具！
    :param root:    数据集存储的目录
    :param mode:    mode:当前加载的数据是train,val,还是test
    :return:
    """
    # # 创建数字编码表,范围0-4;
    # name2label = {}  # "sq...":0   类别名:类标签;  字典 可以看一下目录,一共有5个文件夹,5个类别：0-4范围;
    # for name in sorted(os.listdir(os.path.join(root))):     # 列出所有目录;
    #     if not os.path.isdir(os.path.join(root, name)):
    #         continue
    #     # 给每个类别编码一个数字
    #     name2label[name] = len(name2label.keys())
 
    # 读取Label信息;保存索引文件images.csv
    # [file1,file2,], 对应的标签[3,1] 2个一一对应的list对象。
    # 根据目录,把每个照片的路径提取出来,以及每个照片路径所对应的类别都存储起来，存储到CSV文件中。
    size = 24
    images,labels,boxes,landmarks = red_tf(data_path,size)
 
    # 图片切割成，训练70%，验证15%，测试15%。
    if mode == 'train':                                                     # 70% 训练集
        images = images[:int(0.7 * len(images))]
        labels = labels[:int(0.7 * len(labels))]
        boxes  = boxes[:int(0.7 * len(boxes))]
        landmarks = landmarks[:int(0.7 * len(landmarks))]
    elif mode == 'val':                                                     # 15% = 70%->85%  验证集
        images = images[int(0.7 * len(images)):int(0.85 * len(images))]
        labels = labels[int(0.7 * len(labels)):int(0.85 * len(labels))]
        boxes = boxes[int(0.7 * len(boxes)):int(0.85 * len(boxes))]
        landmarks = landmarks[int(0.7 * len(landmarks)):int(0.85 * len(landmarks))]
    else:                                                                   # 15% = 70%->85%  测试集
        images = images[int(0.85 * len(images)):]
        labels = labels[int(0.85 * len(labels)):]
        boxes = boxes[int(0.85 * len(boxes)):]
        landmarks = landmarks[int(0.85 * len(landmarks)):]
    ima = tf.data.Dataset.from_tensor_slices(images)
    lab = tf.data.Dataset.from_tensor_slices(labels)
    roi = tf.data.Dataset.from_tensor_slices(boxes)
    land = tf.data.Dataset.from_tensor_slices(landmarks)
 
    #train_data = tf.data.Dataset.zip((ima, lab, roi,land)).shuffle(1000).batch(16)
    train_data = tf.data.Dataset.zip((ima, lab, roi,land)).batch(16)
    train_data = list(train_data.as_numpy_iterator())
    return train_data

# 图像色相变换
def image_color_distort(inputs):
    inputs = tf.image.random_contrast(inputs, lower=0.5, upper=1.5)
    inputs = tf.image.random_brightness(inputs, max_delta=0.2)
    inputs = tf.image.random_hue(inputs,max_delta= 0.2)
    inputs = tf.image.random_saturation(inputs,lower = 0.5, upper= 1.5)
    return inputs

In [21]:
def train(eopch):
    model = Rnet()
    #model.load_weights("rnet.h5")
 
    optimizer = keras.optimizers.Adam(learning_rate=1e-3)
    off = 1000
    acc_meter = metrics.Accuracy()
    for epoch in tqdm(range(eopch)):
 
        for i,(img,lab,boxes,landmarks) in enumerate(load_pokemon("train")):
 
 
            img = image_color_distort(img)
            # 开一个gradient tape, 计算梯度
            with tf.GradientTape() as tape:
                print(lab)
                cls_prob, bbox_pred, landmark_pred = model(img)
                cls_loss = cls_ohem(cls_prob, lab)
                bbox_loss = bbox_ohem(bbox_pred, boxes,lab)
                landmark_loss = landmark_ohem(landmark_pred, landmarks, lab)
                # accuracy = cal_accuracy(cls_prob, label_batch)
 
 
                total_loss_value = cls_loss + 0.5 * bbox_loss + 0.5 * landmark_loss
                grads = tape.gradient(total_loss_value, model.trainable_variables)
                optimizer.apply_gradients(zip(grads, model.trainable_variables))
            if i % 200 == 0:
                print('Training loss (for one batch) at step %s: %s' % (i, float(total_loss_value)))
                print('Seen so far: %s samples' % ((i + 1) * 16))
 
 
        for i, (v_img, v_lab1, boxes, landmarks) in enumerate(load_pokemon("val")):
            v_img = image_color_distort(v_img)
            with tf.GradientTape() as tape:
                cls_prob, bbox_pred, landmark_pred = model(v_img)
                cls_loss = cls_ohem(cls_prob, v_lab1)
                bbox_loss = bbox_ohem(bbox_pred, boxes,v_lab1)
                landmark_loss = landmark_ohem(landmark_pred, landmarks, v_lab1)
                # accuracy = cal_accuracy(cls_prob, label_batch)
 
 
                total_loss_value = cls_loss + 0.5 * bbox_loss + 0.5 * landmark_loss
                grads = tape.gradient(total_loss_value, model.trainable_variables)
                optimizer.apply_gradients(zip(grads, model.trainable_variables))
            if i % 200 == 0:
                print('val___ loss (for one batch) at step %s: %s' % (i, float(total_loss_value)))
                print('Seen so far: %s samples' % ((i + 1) * 16))
    model.save_weights('./Weights/Rnet_wight/rnet_30.ckpt')

In [22]:
train(30)

  0%|          | 0/30 [00:00<?, ?it/s]

[-2. -1. -1. -2.  0.  1. -2. -2. -1.  1. -1.  1. -2. -1.  0.  1.]
Training loss (for one batch) at step 0: 2.540825366973877
Seen so far: 16 samples
[ 1.  1. -2.  1.  1. -1. -1. -2. -1.  1. -2.  1. -2.  1. -2.  0.]
[ 1.  1.  0.  0. -2.  0. -1.  1.  1.  1. -2.  1. -1. -2.  0. -2.]
[ 0.  0.  1. -2.  1. -1. -2. -2.  1.  0.  1.  1. -1.  0.  1.  0.]
[-1. -2. -1. -2. -1. -1. -1.  1.  1.  1.  0. -2.  1.  0. -2.  1.]
[ 1.  1. -1. -1.  1.  0. -2.  1. -2. -1. -2.  1.  0.  1. -1.  1.]
[ 1. -2. -2.  0.  0. -2. -2. -1.  0. -2.  0. -1. -2.  0. -2. -2.]
[ 0. -1.  1.  0.  1.  1.  1.  1. -1. -2.  1. -1. -2.  1.  1. -1.]
[-1.  1. -1.  0.  1.  1.  0. -1.  0.  0.  0. -1. -1.  0. -2. -2.]
[ 1. -1. -2. -2.  0. -1.  1. -2. -2. -2.  1. -2. -2.  0. -1.  1.]
[ 1. -2. -2.  0.  0. -2. -2.  1.  0.  0.  1. -2.  1.  0. -2.  1.]
[ 1.  1. -1.  0.  1. -1.  0.  1. -2.  1.  0.  1. -1.  1. -1. -1.]
[-1. -2. -1. -1. -2. -1.  0. -2. -2.  0. -1.  0. -2. -2. -2.  1.]
[ 1.  0.  0. -2. -1. -1.  1. -1. -2. -2.  0.  0. -2.  0.  0

[ 1.  0. -1. -1.  0. -1.  1.  1.  1.  0.  1. -1. -1.  0.  0. -2.]
[-2. -1.  0.  1. -2.  0. -2.  0. -2.  1. -2. -2. -1. -2.  0.  1.]
[-2. -1.  0.  1.  0. -2. -1. -2. -1. -1. -2. -2. -1.  1. -1.  0.]
[ 1. -2.  0. -1.  0.  1.  1. -1. -1.  1. -1. -2.  1.  0.  0.  1.]
[ 0. -2.  0.  1.  0.  1.  1.  0.  1.  1. -2. -2.  1. -2.  0.  1.]
[-1. -1.  1. -1.  0. -2.  0.  0. -1.  0.  0.  1.  1.  1. -2.  0.]
[-1. -2. -1.  0. -1. -2.  1.  0.  0. -1. -1.  1. -1. -1.  0.  1.]
[ 1.  1.  1.  1. -2.  0.  0.  0. -2.  1.  0. -2. -1.  1. -1. -2.]
[ 1.  1. -2. -2.  0. -1. -2. -2.  1.  0.  0.  1.  1.  0. -1.  1.]
[ 0. -1.  0.  1. -1.  1. -1. -2.  0.  0.  0. -2. -1.  1. -1. -1.]
[ 0.  1. -1. -1.  1.  1.  1. -1.  0.  0. -1. -1.  0.  0.  0.  1.]
[ 0. -1. -2. -1. -1. -1.  0.  1.  0.  1.  1.  0.  0.  1. -2.  1.]
[ 0. -1. -1.  1. -1.  0.  0.  0. -1. -1.  1. -2.  0. -2. -1. -1.]
[ 0.  0. -1.  0. -2.  0. -2.  1. -2. -2.  1. -1.  0.  1.  1. -2.]
[-1.  1.  0. -1.  0. -1.  0.  0.  0.  0.  0.  1.  1. -1. -1.  1.]
[-1. -2.  

[ 1. -2.  1.  1. -1.  1.  1. -1. -2. -1.  0. -2. -2. -2. -2. -1.]
[-1. -1. -2.  1. -2. -1.  1. -1.  1. -2.  1.  1. -1. -1. -1.  1.]
[ 0. -1.  1. -2.  1. -1. -1. -1. -1. -2. -2.  0. -2.  1. -1.  0.]
[ 1. -1.  1.  0. -1.  1.  1. -1.  0.  1.  1. -1.  1. -2. -2. -1.]
[-1. -2. -1. -2.  0. -1.  0.  1.  0. -2. -2.  1.  0. -1. -1.  1.]
[ 0. -2.  0.  1. -2.  1.  1.  1.  1.  0.  0.  1. -1. -2.  1. -1.]
[ 0. -1.  1. -2. -1.  0. -1. -1.  0.  1.  1.  0. -2.  0. -1. -2.]
[-2.  1. -2.  1. -1. -1. -2.  0.  1. -2.  0. -2.  1.  0. -1. -1.]
[-2. -1.  0.  1. -2.  1.  1.  1. -1. -1.  0. -2.  1. -1. -2. -2.]
[ 1. -1. -2.  1. -1.  0.  1.  1.  1. -1.  1. -1. -2.  1.  1.  1.]
[ 1.  1.  1. -2. -2.  0.  1. -2. -2.  1. -1.  1. -1.  1.  1.  0.]
[ 0. -2. -1.  1. -2. -2.  0. -1.  1.  1. -1.  1. -1.  0.  1. -1.]
[-1. -1. -1.  0. -1.  0.  1.  0.  0. -2.  0. -1. -2.  0. -1. -1.]
[ 0.  1. -1. -2. -2.  1.  1.  0. -2. -1. -1.  1. -1.  1.  1.  0.]
[ 1.  1. -2. -2.  1.  1.  1.  1.  0.  1.  1. -1.  0.  0.  1.  0.]
[ 1. -1. -

[-1. -2. -2.  1.  1. -2.  1. -2.  0.  0.  1.  0.  0.  0.  0.  1.]
[-2.  0. -2. -1.  1. -2. -2. -1.  1. -2.  0. -1. -1. -2.  1. -2.]
[-2. -2. -2. -1. -1.  0. -2. -1. -2.  0. -1. -1.  1.  1. -1. -1.]
[ 1. -1. -1. -2. -1. -1.  1.  1. -2.  1.  1. -2. -2. -1. -2.  1.]
[-1.  0. -1.  0.  0.  0.  0.  0.  1.  1. -2. -2.  1. -1. -2. -1.]
[-2. -2.  0. -1. -1.  0. -1. -1. -2. -2. -2.  1. -1. -2.  1.  0.]
[-1.  0.  1. -1.  1.  0.  0.  0. -2. -2.  1. -2. -1. -2.  1. -1.]
[-2.  0.  0. -2. -2.  0.  0.  1. -1.  0. -1. -1.  1.  1. -2.  0.]
[-2. -1.  0.  0.  1.  1. -1. -1.  0.  0. -2. -2. -1.  1. -2. -2.]
[-1.  1. -1.  0. -2. -2. -1. -2.  1. -1.  0.  1.  1.  1. -1. -1.]
[ 0.  0. -1.  0. -1.  0. -2.  1. -1. -2. -2.  1.  0.  1.  1. -1.]
[-1. -1.  1.  0. -1.  1. -1.  0.  1.  1.  1.  1.  0.  0.  1.  1.]
[-2. -1.  0.  0. -2.  0.  1.  0. -1. -2.  1. -2. -1.  1. -1.  0.]
[ 0.  1.  0. -1. -2. -2. -1.  1. -2. -1.  0. -1.  1. -2. -1. -2.]
[ 1. -1. -1.  0. -1.  1.  0.  1. -2.  1.  0. -2.  1.  0. -2. -2.]
[ 1.  0. -

...

[ 1. -1.  0.  1. -1.  1.  0. -2.  1. -1.  1. -1. -1.  0. -2. -2.]
[-2. -2. -2. -1.  0.  0. -2. -2. -1. -1.  0. -2.  0.  0.  0.  0.]
[ 1. -2. -1. -1.  1.  0.  1. -2.  1. -2.  0.  0.  1.  0. -1.  0.]
[-2. -1.  1.  1. -2. -2. -2. -2.  1. -2.  1.  0. -2. -1.  0.  1.]
[ 1. -2.  1.  0. -2. -1. -1.  0.  0. -1. -2. -1.  0. -2. -2. -1.]
[-2. -1. -1. -1. -2. -1.  0.  1.  1.  1. -2.  0. -1.  1. -1.  0.]
[-2.  0.  1.  0.  0.  1. -2. -2.  0.  0.  0. -2.  0. -2. -1. -1.]
[ 0.  0. -2. -1.  1.  1.  0.  1.  0. -2.  0. -1.  1.  1.  1. -1.]
[-2.  0.  0.  1. -2.  0.  1.  0.  1.  0.  0. -1.  1.  1.  1. -1.]
[-1. -1.  0. -1.  0. -1.  1. -1.  0. -2.  0. -1. -2. -1.  0.  1.]
[ 1.  1. -1. -2. -2.  1.  1.  0.  0. -1. -2.  1.  1. -2.  1.  0.]
[ 1.  0.  0.  0. -2. -1. -2.  1. -1.  1. -2. -1.  0.  1. -1. -1.]
[ 0. -1.  1.  0. -1.  0.  1. -2. -2.  1. -2. -2. -1.  1. -1. -2.]
[-1.  1.  0. -1. -1. -2.  0. -1. -2.  1.  1.  1.  0. -1.  1. -2.]
[-2.  1.  1. -1.  0. -2.  1.  1. -1. -2.  0. -1.  0.  1. -2.  0.]
[-2. -1.  

[ 0. -1.  1.  1.  0.  1. -2.  0.  0. -2.  0. -2. -1.  1. -2. -2.]
[-1. -2. -2. -2. -1.  0. -1.  1. -2.  0.  1. -1. -2.  1.  1. -1.]
[ 1. -2. -2. -2. -1.  0.  1. -1.  1. -2. -1. -1. -1. -2.  1.  1.]
[-2.  0. -1. -1.  0. -1. -2. -1. -2.  0.  0.  1. -1. -2. -2. -1.]
[ 1.  0. -2. -1. -1. -1. -2. -1.  0.  0. -1. -2. -1. -1. -2.  0.]
[-2.  1.  1. -2. -1.  0.  0.  0. -2. -1.  0.  1.  1. -2. -1.  1.]
[ 0. -1. -1. -2. -1. -1.  1.  1. -2.  1.  0.  0.  1.  0.  1.  0.]
[ 1.  0.  0. -2.  0. -1. -2.  0.  0. -2. -2. -2.  0. -1.  1. -1.]
[-2. -1.  0. -1. -2.  1.  0.  1.  1. -2.  0.  0.  0. -2. -2.  1.]
[ 0. -1. -1.  1. -1. -2.  1. -1. -1.  1. -2. -1.  1. -1.  1. -1.]
[ 0.  0.  1.  0. -2.  1.  0.  1.  1.  0.  1.  1.  0. -1. -2. -1.]
[-1.  1. -2.  0. -1. -2. -1.  1. -1. -1. -2.  0. -1. -2. -1.  1.]
[-1. -1.  0.  1. -2. -2.  0.  0.  0. -1. -1.  0.  0.  0. -2. -2.]
[ 1. -2. -1.  1.  1.  1. -2. -2.  1.  1.  1. -2.  1. -1.  1.  0.]
[ 1. -1. -2. -2. -1.  0.  1.  1.  0. -1. -2.  0. -1. -2. -1.  1.]
[-1.  0. -

[-1.  0. -1.  0.  1.  1.  1.  0. -2.  0. -2. -2. -2. -2. -1. -1.]
[-2.  0. -2. -1. -1. -2.  0. -1.  1. -2. -1. -1. -2. -2.  0.  1.]
[-2.  0. -1.  1. -2.  0.  0.  1. -2. -1. -2. -2.  0.  1. -2.  0.]
[-1. -1.  1.  1. -1.  1.  0.  0. -1.  1.  0. -1.  0. -2. -1.  0.]
[-2. -1. -2. -2.  1.  1.  1.  1.  0. -2.  0. -2.  1. -2.  0.  0.]
[-2. -1.  1. -2.  0.  1.  0. -2.  1. -2.  1. -2. -2. -2. -1.  0.]
[-2. -1.  0.  1.  1.  1.  0.  0.  0. -1. -2.  1. -1.  1.  1. -1.]
[-2. -1.  1.  0. -2.  0.  1. -1. -1.  0. -1. -1. -2. -2. -1.  0.]
[ 1.  0.  0.  1.  0. -2.  0.  0.  1.  0. -1.  1. -1.  1.  0. -1.]
[-2. -1. -2. -2. -1.  1.  1. -1. -2. -2.  1.  1. -1. -2. -1.  0.]
[ 0.  0.  0. -1.  0. -2.  0. -2.  0. -1. -2. -1.  0. -2.  0.  1.]
[-1. -1. -2. -1. -2.  1.  1.  0. -2.  1. -1. -2.  1.  1. -1. -2.]
[ 1.  1.  1. -1.  1.  1.  0.  0. -1.  1. -1. -1. -2. -2. -2.  0.]
[-1. -2. -2. -2. -2. -1. -1. -2.  1. -1.  0.  1.  1. -1.  0. -1.]
[-2.  0.  1.  1.  0. -1.  1.  1.  1. -2.  1. -1. -1.  1. -2.  0.]
[ 0. -2. -

[ 1.  1.  1.  1. -1. -1. -1.  1. -2. -1.  0.  1.  1.  1.  1.  0.]
[ 0. -1.  1.  0.  0.  0.  1.  1. -2.  1.  1.  1.  0. -2.  0.  0.]
[ 1.  0.  0.  1. -1.  1. -2.  0. -1.  0. -2.  0.  1.  1.  1. -2.]
[ 0.  1. -2. -2. -1.  1.  0.  1.  1. -1.  1. -1. -1. -1.  0.  1.]
[ 0. -2. -2.  0. -2.  1. -2. -2. -2.  0. -1. -1.  0. -1.  1.  0.]
[-1. -1.  1.  0.  1.  1.  0.  0. -2.  1.  1. -2.  1.  1.  1.  0.]
[-1.  1.  0.  0.  1. -2.  1.  1.  1. -1. -2.  1.  1. -2. -1.  1.]
[-1.  1.  1. -1. -2.  1. -2. -2. -1. -1. -1. -1. -1.  0. -2. -2.]
[-2. -1. -1.  1.  1. -2. -2.  0. -2.  1. -1. -2. -1.  0.  1.  1.]
[ 0. -2. -2. -2. -1. -2. -2. -2. -1. -2. -2.  0. -1.  1.  0. -1.]
[ 1. -2. -1.  0.  1. -2. -1. -2. -2. -1.  1.  0.  0. -1. -2. -2.]
[ 0.  1.  1. -1. -2.  0. -2.  0.  0.  0. -1.  1.  1. -2.  1.  1.]
[-1.  0. -1. -2. -1.  0.  0. -2.  1. -1. -1. -2. -1. -2.  0.  1.]
[-2. -1.  0. -2. -1. -2. -1. -2. -1. -1.  1.  0. -1.  1.  1.  1.]
[ 1.  0.  1.  1.  1.  0. -2. -1.  0. -2.  1. -2.  1. -1. -2. -2.]
[ 0. -1. -

100%|██████████| 30/30 [42:52<00:00, 85.76s/it]
